In [1]:
pip install pymupdf pandas streamlit openai

  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyarrow-20.0.0-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached charset_normalizer-3.4.2-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import fitz  # PyMuPDF

def extract_pdf_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_path = r"C:\Users\Dell\VS_code\Final_Project\statement.pdf"
raw_text = extract_pdf_text(pdf_path)

# Show first few lines
print(raw_text[:1000])

Ms  Savundariya D/o Ramanathan
  
Statement From : 18/03/25    TO : 17/04/25
Account Branch
A/C Open Date
Limit
City
Phone No.
Cust ID
Account Status
Account number
State
RTGS/NEFT IFSC
JOINT HOLDERS:
Address
Email
Currency
Pr.Code
Br.Code
OTHER
: Nungambakkam
: No.40,
  Nungambakkam High Road
  (n.h.road)
: Chennai 600034
: Tamil Nadu
: 18002600/18001600
: HDFC0000082
: RMSAVUNDARIYA@GMAIL.COM
: 0.00
: INR
: 217156093
: 161
:82
: 50100577266913
: 16/12/2022
: Account Open Regular
Not Applicable
Not Applicable
Nomination     : Registered
600240006
MICR :
259 Periyar Nagar Pudukkottai
Pudukkottai
Tamil Nadu
India
622003
Expected AMB : 0.00
Date
Narration
Chq. / Ref No.
Value Date
Withdrawal Amount
Deposit Amount
Closing Balance*
20/03/2025
UPI-BLINKIT-BLINKIT104020.RZP@RXAIRTEL-A
IRP0000011-101788754597-BLINKIT
101788754597
20/03/2025
361.00
0.00
88,252.19
22/03/2025
UPI-BLINKIT-BLINKIT104020.RZP@RXAIRTEL-A
IRP0000011-101870665917-BLINKIT
101870665917
22/03/2025
316.00
0.00
87,936.19
25

In [5]:
import re
import pandas as pd

def parse_multiline_transactions(text):
    lines = text.split('\n')
    data = []
    i = 0

    while i < len(lines):
        line = lines[i].strip()

        # Look for line starting with DD/MM/YYYY
        if re.match(r'\d{2}/\d{2}/\d{4}', line):
            date = line  # Example: 20/03/2025
            desc_lines = []
            i += 1

            # Collect description lines until we hit an amount line
            while i < len(lines) and not re.match(r'^\d{1,3}(,\d{3})*(\.\d{2})?$', lines[i].strip()):
                if lines[i].strip():  # Ignore empty lines
                    desc_lines.append(lines[i].strip())
                i += 1

            # Get amount (withdrawal or deposit)
            if i < len(lines):
                amount_line = lines[i].strip()
                try:
                    amount = float(amount_line.replace(",", ""))
                except:
                    amount = 0.0
                i += 1  # Skip to next line

                # Optional: Skip deposit/closing balance lines
                i += 2

                description = " ".join(desc_lines)
                data.append([date, amount, description])
        else:
            i += 1

    df = pd.DataFrame(data, columns=["Date", "Amount", "Description"])
    return df

# Run the updated parser
df = parse_multiline_transactions(raw_text)

# Save as CSV
df.to_csv("parsed_transactions.csv", index=False)

# Show sample
print(df.head())

         Date   Amount                                        Description
0  20/03/2025   361.00  UPI-BLINKIT-BLINKIT104020.RZP@RXAIRTEL-A IRP00...
1  22/03/2025   316.00  UPI-BLINKIT-BLINKIT104020.RZP@RXAIRTEL-A IRP00...
2  25/03/2025  1050.00  UPI-REDBUS INDIA PRIVATE-REDBUS1ONLINE.G PAY@O...
3  25/03/2025   840.00  UPI-REDBUS INDIA PRIVATE-REDBUS1ONLINE.G PAY@O...
4  27/03/2025   534.69  UPI-BIGBASKET-BIGBASKET1@PTYBL-YESB0PTMU PI-50...


In [6]:
def categorize(description):
    desc = description.lower()
    if "swiggy" in desc or "zomato" in desc or "blinkit" in desc or "zepto" in desc:
        return "Food Delivery"
    elif "redbus" in desc or "uber" in desc or "ola" in desc:
        return "Travel"
    elif "jio" in desc or "recharge" in desc:
        return "Recharge"
    elif "insurance" in desc:
        return "Insurance"
    elif "amazon" in desc:
        return "Shopping"
    elif "life science" in desc:
        return "Healthcare"
    elif "grand fresh" in desc or "bigbasket" in desc:
        return "Groceries"
    elif "maintenance" in desc:
        return "Utilities"
    else:
        return "Others"

# Apply category to each row
df["Category"] = df["Description"].apply(categorize)

# Save again
df.to_csv("categorized_transactions.csv", index=False)

# Show with categories
print(df.head())

         Date   Amount                                        Description  \
0  20/03/2025   361.00  UPI-BLINKIT-BLINKIT104020.RZP@RXAIRTEL-A IRP00...   
1  22/03/2025   316.00  UPI-BLINKIT-BLINKIT104020.RZP@RXAIRTEL-A IRP00...   
2  25/03/2025  1050.00  UPI-REDBUS INDIA PRIVATE-REDBUS1ONLINE.G PAY@O...   
3  25/03/2025   840.00  UPI-REDBUS INDIA PRIVATE-REDBUS1ONLINE.G PAY@O...   
4  27/03/2025   534.69  UPI-BIGBASKET-BIGBASKET1@PTYBL-YESB0PTMU PI-50...   

        Category  
0  Food Delivery  
1  Food Delivery  
2         Travel  
3         Travel  
4      Groceries  


In [7]:
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
df["Month"] = df["Date"].dt.strftime("%b-%Y")

monthly_spending = df.groupby("Month")["Amount"].sum()
print("📆 Monthly Spending:\n", monthly_spending)

📆 Monthly Spending:
 Month
Apr-2025    9298.44
Mar-2025    6913.17
Name: Amount, dtype: float64


In [8]:
category_spending = df.groupby("Category")["Amount"].sum().sort_values(ascending=False)
print("🏷 Category-wise Spending:\n", category_spending)

🏷 Category-wise Spending:
 Category
Others           7240.78
Food Delivery    3380.11
Insurance        1900.82
Travel           1890.00
Groceries        1349.90
Recharge          448.00
Shopping            2.00
Name: Amount, dtype: float64


In [19]:
pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
# Convert monthly and category spending to strings with ₹ symbol

monthly_spending_str = "\n".join(
    [f"{month}    ₹{amount:,.2f}" for month, amount in monthly_spending.items()]
)

category_spending_str = "\n".join(
    [f"{category}: ₹{amount:,.2f}" for category, amount in category_spending.items()]
)

prompt = f"""
You are a financial advisor. Based on the following monthly and category-wise spending in Indian Rupees (₹), provide 3 personalized financial tips.

Monthly Spend:
{monthly_spending_str}

Category-wise Spend:
{category_spending_str}
"""

# --------------------
# Step 6: Call OpenRouter/OpenAI with the prompt

client = openai.OpenAI(
    api_key="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",  # <-- UPDATE your API key here
    base_url="https://openrouter.ai/api/v1"
)

response = client.chat.completions.create(
    model="openrouter/auto",  # Reliable default OpenRouter model
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print("💡 Financial Advice:\n")
print(response.choices[0].message.content)

💡 Financial Advice:

Here are 3 personalized financial tips based on your spending patterns:

### 1. **Audit the "Others" Category**  
   - **Issue**: Your "Others" category (₹7,240.78) accounts for **~60% of total spending** over two months, indicating unplanned/unclassified expenses.  
   - **Action**: Track this category rigorously for 1-2 months using a budgeting app. Label these expenses (e.g., entertainment, gifts, impulse purchases) to identify leaks. Even a 20% reduction here could save **₹1,448/month**.

### 2. **Reduce Food Delivery Costs**  
   - **Issue**: Food delivery (₹3,380.11) is **2.5x higher than groceries** (₹1,349.90), suggesting frequent reliance on convenience meals.  
   - **Action**: Shift to weekly meal preps using groceries. For example, replacing 4 delivery meals/week with homemade options could save **₹1,500–2,000/month** while improving nutrition.

### 3. **Stabilize Monthly Spending**  
   - **Issue**: Spending spiked by **34% in April 2025** (₹9,298 vs. 